In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random as rand
import numpy as np
from network import Network
from util import idem, one, sigmoid, derivative_sigmoid as der_sigm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
%matplotlib inline
sns.set()

In [ ]:
def plot(pontos, a, b):
    px = [p[0] for p in pontos]
    py = [p[1] for p in pontos]
    y = [a*x + b for x in px]
    plt.figure(figsize=(8,6))
    plt.plot(px, py, 'g.')
    plt.plot(px, y, 'r--')
    plt.show()

In [ ]:
def reta(ponto):
    return ponto[0] + ponto[1] - 10

def sinal(ponto):
    return 1 if reta(ponto) >= 0 else 0

x_train, y_train = [], []
for i in range(1000):
    x_train.append([rand.random()*50-25, rand.random()*50-25])
    y_train.append([sinal(x_train[i])])
print(y_train[:10])

In [ ]:
network = Network([2, 1, 1], taxa=0.08, ativacoes=[sigmoid, der_sigm, idem, one])
a = network.layers[2].neurons[0].weights[0]
b = network.layers[2].neurons[0].bias
mse = network.norma_l2(x_train, y_train)
print("Início: %f*x + %f => MSE = %f"%(a, b, mse))
plot(x_train, a, b)

In [ ]:
for k in tqdm(range(100)):
    network.train(x_train, y_train)

In [ ]:
for layer in network.layers[1:]:
    for neuron in layer.neurons:
        print(neuron.weights, neuron.bias)
        
a = network.layers[1].neurons[0].weights[0]
b = network.layers[1].neurons[0].bias
mse = network.norma_l2(x_train, y_train)
print("Após %d passos: %f*x + %f => MSE = %f"%(k, a, b, mse))
plot(x_train, -a, -b)

In [ ]:
def sinal_reverso(num):
    return 1 if num[0] >= 0.5 else 0

network.predict([[1, 1], [-1, 1], [1, -1], [-1, -1], [0, 0]], sinal_reverso)

Me parece que o esquema será gerar um framework pra otimização linear dentro da própria classe network, pq a classe perceptron já tá muito bem feita para o caso não-linear, e com ativação por sinal

In [ ]:
from perceptron import Perceptron
from scores import Scores

In [ ]:
def sinal(ponto):
    return "1 Quadrante" if ponto[0] >= 0 and ponto[1] >= 0 else "Outros"

x_train, y_train = [], []
for i in range(1000):
    x_train.append([rand.random()*50-25, rand.random()*50-25])
    y_train.append([sinal(x_train[i])])
print(x_train[:10])
print(y_train[:10])

In [ ]:
perceptron = Perceptron(ativacao="sigm", M=20)
perceptron.tqdm = tqdm
perceptron.ativacao_saida = idem
perceptron.der_ativacao_saida = one

In [ ]:
perceptron.treinar(np.array(x_train), y_train)
print("Estrutura da rede:", perceptron.network.estrutura)

In [ ]:
# exibindo pesos e viéses das camadas, exceto de entrada
for layer in perceptron.network.layers[1:]:
    for neuron in layer.neurons:
        print(neuron.weights, neuron.bias)

In [ ]:
saidas = perceptron.processar([[1, 1], [-1, 1], [1, -1], [-1, -1]])
for s in saidas:
    print(s)
previsoes = perceptron.prever([[1, 1], [-1, 1], [1, -1], [-1, -1]])
for p in previsoes:
    print(p)

In [ ]:
x_test, y_test = [], []
for i in range(1000):
    x_test.append([rand.random()*50-25, rand.random()*50-25])
    y_test.append([sinal(x_test[i])])

In [ ]:
y_pred = perceptron.prever(np.array(x_test))
mse = perceptron.funcao_erro(np.array(x_test), y_test)
score = Scores(y_test, y_pred)
score.exibir_grafico("MSE: %f"%mse)